In [ ]:
#Cell [0]

# Standard Libraries
import os  # For file system operations such as creating directories.
import warnings  # For controlling warning messages.
from datetime import timedelta  # For handling time-related operations.

# Third-party Libraries

# Joblib for saving/loading models
import joblib  # For serializing Python objects (like models).

# Numpy and Pandas for numerical and data manipulation
import numpy as np  # For numerical computations, especially with arrays.
import pandas as pd  # For data manipulation and dataframes.

# Seaborn and Matplotlib for visualizations
import seaborn as sns  # Advanced data visualization library based on matplotlib.
import matplotlib.pyplot as plt  # Basic plotting library.

# Scipy for statistical distributions (used in RandomizedSearchCV)
from scipy.stats import randint, uniform, loguniform  # For specifying parameter search distributions.

# Sklearn utilities and transformers
from sklearn.base import BaseEstimator, TransformerMixin  # Base classes for creating custom transformers/estimators.
from sklearn.compose import ColumnTransformer  # For applying different preprocessing pipelines to specific columns.
from sklearn.pipeline import Pipeline, make_pipeline  # For creating machine learning workflows.

# Sklearn preprocessors and imputers
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler  # For encoding, feature generation, and scaling.
from sklearn.impute import SimpleImputer  # For handling missing values in datasets.

# Sklearn model evaluation and hyperparameter tuning
from sklearn.model_selection import (KFold, RandomizedSearchCV, cross_val_predict, cross_val_score, train_test_split)  # For cross-validation, hyperparameter tuning, and splitting datasets.
from sklearn.metrics import mean_squared_error  # For evaluating model performance with error metrics like MSE.

# Sklearn models
from sklearn.linear_model import BayesianRidge, Ridge, LinearRegression, Lasso  # Linear models for regression tasks.
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor  # Ensemble models for regression.
from sklearn.svm import SVR  # Support Vector Regression.
from sklearn.neural_network import MLPRegressor  # Neural network regressor model.

# XGBoost and LightGBM - Gradient Boosting models
from xgboost import XGBRegressor  # XGBoost regressor for gradient boosting.
from lightgbm import LGBMRegressor  # LightGBM regressor for fast gradient boosting.

# Warnings related to model convergence
from sklearn.exceptions import ConvergenceWarning  # To suppress warnings related to non-convergence.

#Cell [1]

# Suppress warnings for cleaner output
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", message="A value is trying to be set on a copy of a DataFrame")
warnings.filterwarnings("ignore", message="No further splits with positive gain")
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Create directories for storing outputs
os.makedirs('figures', exist_ok=True)  # For storing plots and visualizations
os.makedirs('models', exist_ok=True)   # For saving trained models
os.makedirs('saved_objects', exist_ok=True)  # For miscellaneous saved objects

# Configure KFold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Function to save models and pipelines
def store_model_or_pipeline(obj, name=""):
    """Store a trained model or pipeline to the 'models' folder."""
    if name == "":
        name = type(obj).__name__
    joblib.dump(obj, f'models/{name}.pkl')
    print(f'{name} saved successfully.')

# Function to load saved models and pipelines
def load_model_or_pipeline(name):
    """Load a model or pipeline from the 'models' folder."""
    obj = joblib.load(f'models/{name}.pkl')
    print(f'{name} loaded successfully.')
    return obj



#Cell [2]

# Load the data and perform initial cleaning
raw_data = pd.read_csv('datasets/NewYork.csv')
raw_data['datetime'] = pd.to_datetime(raw_data['datetime'])
raw_data.drop(columns=["name", "icon", "stations", "description"], inplace=True)

# Check for missing values before imputation
print("\nMissing values before imputation:")
print(raw_data.isnull().sum())

# Define function to remove outliers using IQR method
def remove_outliers(df, column, factor=1.5):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - factor * IQR
    upper_bound = Q3 + factor * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Remove outliers from 'tempmax' column
raw_data = remove_outliers(raw_data, 'tempmax')
print("\nShape of data after removing outliers:", raw_data.shape)

# Impute missing values
for column in raw_data.columns:
    if raw_data[column].dtype == 'object':
        raw_data[column].fillna('Unknown', inplace=True)  # For categorical columns
    else:
        raw_data[column].fillna(raw_data[column].median(), inplace=True)  # For numerical columns

# Check for missing values after imputation
print("\nMissing values after imputation:")
print(raw_data.isnull().sum())




#Cell [3]

print('\n____________ Dataset info ____________')
print(raw_data.info())              
print('\n____________ Some first data examples ____________')
print(raw_data.head(3)) 
print('\n____________ Statistics of numeric features ____________')
print(raw_data.describe())    





#Cell [3.1]

# Create correlation matrix for numeric features
plt.figure(figsize=(12, 10))
numeric_data = raw_data.select_dtypes(include=[np.number])
corr = numeric_data.corr()

# Handle any non-finite values in the correlation matrix
if not np.isfinite(corr.values).all():
    print("Warning: Correlation matrix contains non-finite values. Replacing with 0.")
    corr_array = np.nan_to_num(corr.values, nan=0, posinf=0, neginf=0)
    corr = pd.DataFrame(corr_array, index=corr.index, columns=corr.columns)

# Create and display heatmap of the correlation matrix
plt.figure(figsize=(12, 8))
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', cbar=True, square=True)
plt.title('Heatmap of Feature Correlations')
plt.tight_layout()

# Save the heatmap as a high-resolution image
plt.savefig('figures/correlation_heatmap.png', format='png', dpi=300)

# Display the plot
plt.show()




#Cell [3.2]

# Select numeric features from the dataset
numeric_features = raw_data.select_dtypes(include=[np.number]).columns
n_features = len(numeric_features)

# Calculate the number of rows needed for the subplot grid
n_rows = (n_features + 1) // 2

# Create a large figure to accommodate all histograms
plt.figure(figsize=(15, 5 * n_rows))

# Create a histogram for each numeric feature
for i, feature in enumerate(numeric_features, 1):
    plt.subplot(n_rows, 2, i)
    sns.histplot(raw_data[feature], kde=True)
    plt.title(f'Distribution of {feature}')

# Adjust the layout and save the figure
plt.tight_layout()
plt.savefig('figures/hist_raw_data.png', format='png', dpi=300)

# Display the plot
plt.show()



#Cell [3.3]

# Select all numeric features except 'tempmax'
numeric_features = [col for col in numeric_features if col != 'tempmax']
n_features = len(numeric_features)

# Calculate the number of rows needed for the subplot grid
n_rows = (n_features + 1) // 2

# Create a large figure to accommodate all scatter plots
plt.figure(figsize=(15, 5 * n_rows))

# Create a scatter plot for each feature vs tempmax
for i, feature in enumerate(numeric_features, 1):
    plt.subplot(n_rows, 2, i)
    sns.scatterplot(data=raw_data, x=feature, y='tempmax', alpha=0.5)
    plt.title(f'Max Temperature vs {feature}')

# Adjust the layout and save the figure
plt.tight_layout()
plt.savefig('figures/scatter_tempmax_vs_features.png', format='png', dpi=300)

# Display the plot
plt.show()


#Cell [3.4]

# Select the temperature-related features for plotting
temp_features = ['temp', 'feelslike', 'tempmax', 'feelslikemax', 'tempmin', 'feelslikemin']

# Create the boxplot
plt.figure(figsize=(12, 8))
sns.boxplot(data=raw_data[temp_features])

# Set titles and labels for the plot
plt.title('Distribution of Temperature Variables')
plt.ylabel('Temperature (°C)')
plt.xticks(ticks=range(len(temp_features)), labels=temp_features)

# Adjust layout and save the plot
plt.tight_layout()
plt.savefig('figures/boxplot_temp_distributions.png', format='png', dpi=300)

# Display the plot
plt.show()




#Cell [4]
class EnhancedFeatureAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_features=True):
        self.add_features = add_features
    
    def fit(self, X, y=None):
        return self
    
    ## Time-based Features
    def transform(self, X):
        X_ = X.copy()
        if isinstance(X_, pd.DataFrame) and self.add_features and 'datetime' in X_.columns:
            # Add new features from datetime
            X_['day_of_year'] = X_['datetime'].dt.dayofyear
            X_['month'] = X_['datetime'].dt.month
            X_['day_of_week'] = X_['datetime'].dt.dayofweek
            X_['is_weekend'] = X_['day_of_week'].isin([5, 6]).astype(int)
            X_['day_of_year_sin'] = np.sin(2 * np.pi * X_['day_of_year'] / 365.25)
            X_['day_of_year_cos'] = np.cos(2 * np.pi * X_['day_of_year'] / 365.25)
            X_.drop(columns=['datetime'], inplace=True)
        return X_
    
    
    
### Lag Features and Rolling Averages
# Function to add lag features
def add_lag_features(df, column, lags):
    for lag in lags:
        df[f'{column}_lag_{lag}'] = df[column].shift(lag)
    return df

# Function to add rolling averages
def add_rolling_features(df, column, windows):
    for window in windows:
        df[f'{column}_rolling_{window}'] = df[column].rolling(window).mean()
    return df



## Feature Selection and Correlation Analysis
# Apply lag features and rolling averages to selected columns
selected_columns = ['tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin', 'feelslike']
for col in selected_columns:
    raw_data = add_lag_features(raw_data, col, lags=[1, 2, 3])
    raw_data = add_rolling_features(raw_data, col, windows=[3, 7])

# Evaluate Feature Correlation
numeric_data = raw_data.select_dtypes(include=[np.number])
correlation_matrix = numeric_data.corr()
tempmax_correlation = correlation_matrix['tempmax'].sort_values(ascending=False)
print("Correlation of features with tempmax:\n", tempmax_correlation)

# Select features based on correlation threshold
correlation_threshold = 0.8
selected_features = tempmax_correlation[abs(tempmax_correlation) > correlation_threshold].index.tolist()
selected_features.remove('tempmax')  # Remove target variable from features

# Exclude specific features
excluded_features = ['solarenergy', 'solarradiation', 'uvindex']
selected_features = [feature for feature in selected_features if feature not in excluded_features]

print(f"Selected features based on correlation (threshold = {correlation_threshold}):\n", selected_features)

# Separate numeric and categorical features
numeric_features = [feature for feature in selected_features if raw_data[feature].dtype in [np.float64, np.int64]]
categorical_features = [feature for feature in selected_features if raw_data[feature].dtype == 'object']

# Define transformers for numeric and categorical features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Define preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Full pipeline with EnhancedFeatureAdder and preprocessor
enhanced_pipeline = Pipeline(steps=[
    ('feature_adder', EnhancedFeatureAdder()),
    ('preprocessor', preprocessor)
])

# Save the full pipeline
store_model_or_pipeline(enhanced_pipeline, name="full_pipeline")




#Cell [5]

# Prepare the data for modeling
X = raw_data.drop('tempmax', axis=1)  # Features
y = raw_data['tempmax']  # Target variable
X_processed = enhanced_pipeline.fit_transform(X, y)  # Apply our preprocessing pipeline

# Split the data into training and test sets
# We use 80% for training and 20% for testing, which is a common split ratio
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

## Model Implementation and Evaluation
# Define a diverse set of models to evaluate
models = {
    'RandomForestReg': RandomForestRegressor(random_state=42),
    'GradientBoostingReg': GradientBoostingRegressor(random_state=42),
    'LGBMReg': LGBMRegressor(random_state=42, verbosity=-1),
    'XGBBoost': XGBRegressor(random_state=42),
    'BayesianRidge': BayesianRidge(),
    'LinearReg': LinearRegression(),
    'Ridge': Ridge(random_state=42),
    'Lasso': Lasso(random_state=42),
    'SVR': SVR(),
    'MLPRegressor': MLPRegressor(random_state=42),
    'PolynomialReg': make_pipeline(PolynomialFeatures(), LinearRegression())
}

# Justification for model selection:
# 1. Tree-based models (RandomForest, GradientBoosting, LightGBM, XGBoost): Good for capturing non-linear relationships and interactions
# 2. Linear models (LinearReg, Ridge, Lasso): Simple, interpretable, and work well if relationships are mostly linear
# 3. BayesianRidge: Combines linear regression with Bayesian inference, good for uncertainty estimation
# 4. SVR: Can capture non-linear relationships using kernel tricks
# 5. MLPRegressor: Neural network that can capture complex patterns
# 6. PolynomialReg: Can capture non-linear relationships by adding polynomial features

### K-Fold Cross-validation
best_model_name = None
best_cross_val_rmse = float('inf')

print('\n____________ K-Fold Cross Validation and Residual Analysis ____________')
for name, model in models.items():
    model_pipeline = Pipeline(steps=[('model', model)])
    
    # Perform K-Fold Cross-validation
    # We use 5-fold CV as it provides a good balance between bias and variance in error estimation
    cv_rmse_scores = -cross_val_score(model_pipeline, X_train, y_train, cv=kfold, scoring='neg_mean_squared_error')
    avg_rmse = np.sqrt(cv_rmse_scores.mean())
    
    # Save the RMSE scores for each fold
    joblib.dump(cv_rmse_scores, f'saved_objects/{name}_rmse.pkl')
    
    print(f'{name:<20} K-Fold Avg RMSE: {avg_rmse:.4f}')
    
    
    ### Residual Analysis
    # Calculate and plot residuals
    y_train_pred = cross_val_predict(model_pipeline, X_train, y_train, cv=kfold)
    residuals = y_train - y_train_pred
    
    plt.figure(figsize=(10, 6))
    sns.histplot(residuals, kde=True, bins=50)
    plt.title(f'Residuals Distribution - {name}')
    plt.savefig(f'figures/residuals_{name}.png', format='png', dpi=300)
    plt.close()
    
    # Update best model if current model performs better
    if avg_rmse < best_cross_val_rmse:
        best_cross_val_rmse = avg_rmse
        best_model_name = name

# Print and store the best model
print(f"\nBest model after K-Fold Cross Validation: {best_model_name} with RMSE: {best_cross_val_rmse:.4f}")
best_model = models[best_model_name]
store_model_or_pipeline(best_model, name=best_model_name)

# Display residual plot for the best model
model_pipeline = Pipeline(steps=[('model', best_model)])
y_train_pred_best = cross_val_predict(model_pipeline, X_train, y_train, cv=kfold)
residuals_best = y_train - y_train_pred_best

plt.figure(figsize=(10, 6))
sns.histplot(residuals_best, kde=True, bins=50)
plt.title(f'Residuals Distribution - {best_model_name}')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.show()

# Justification for using RMSE as the primary metric:
# 1. RMSE is in the same unit as the target variable (temperature), making it interpretable
# 2. It penalizes large errors more heavily, which is important for temperature prediction
# 3. It's widely used in regression problems, allowing for easy comparison with other models or studies

# Justification for using K-fold Cross-Validation:
# 1. Provides a more robust estimate of model performance by using all data for both training and validation
# 2. Helps to detect overfitting by showing how well the model generalizes to unseen data
# 3. Reduces the impact of data splitting randomness on model evaluation



#Cell [6]

print(f'\n____________ Fine-tuning the best model: {best_model_name} ____________')

warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Ensure input data is scaled
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

param_grids = {
    'RandomForestReg': {
        'n_estimators': randint(100, 2000),
        'max_depth': randint(5, 50),
        'min_samples_split': randint(2, 20),
        'min_samples_leaf': randint(1, 20),
        'max_features': uniform(0.1, 0.9)
    },
    'GradientBoostingReg': {
        'n_estimators': randint(100, 2000),
        'learning_rate': uniform(0.01, 0.2),
        'max_depth': randint(3, 20),
        'min_samples_split': randint(2, 20),
        'min_samples_leaf': randint(1, 20),
        'subsample': uniform(0.5, 0.5)
    },
    'LGBMReg': {
        'num_leaves': randint(20, 200),
        'learning_rate': uniform(0.01, 0.2),
        'n_estimators': randint(100, 2000),
        'min_child_samples': randint(1, 50),
        'subsample': uniform(0.5, 0.5),
        'colsample_bytree': uniform(0.5, 0.5),
        'verbosity': [-1]
    },
    'XGBBoost': {
        'n_estimators': randint(100, 2000),
        'learning_rate': uniform(0.01, 0.2),
        'max_depth': randint(3, 20),
        'min_child_weight': randint(1, 10),
        'subsample': uniform(0.5, 0.5),
        'colsample_bytree': uniform(0.5, 0.5),
        'gamma': uniform(0, 0.5)
    },
    'BayesianRidge': {
        'alpha_1': uniform(0.001, 1),
        'alpha_2': uniform(0.001, 1),
        'lambda_1': uniform(0.001, 1),
        'lambda_2': uniform(0.001, 1)
    },
    'LinearReg': {
        'fit_intercept': [True, False],
        'copy_X': [True, False],
        'positive': [True, False]
    },
    'Ridge': {
        'alpha': loguniform(1e-3, 1e2),
        'max_iter': [5000, 10000]
    },
    'Lasso': {
        'alpha': loguniform(1e-3, 1e2),
        'max_iter': [5000, 10000]
    },
    'SVR': {
        'C': loguniform(1e-2, 1e2),
        'epsilon': loguniform(1e-3, 1),
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
    },
    'MLPRegressor': {
        'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,), (100,100), (100,50,100)],
        'activation': ['tanh', 'relu', 'logistic'],
        'solver': ['sgd', 'adam'],
        'alpha': loguniform(1e-4, 1e-1),
        'learning_rate': ['constant','adaptive'],
        'learning_rate_init': loguniform(1e-4, 1e-1),
        'max_iter': [200, 500, 1000],
        'early_stopping': [True, False],
        'momentum': uniform(0.0, 1.0),
        'nesterovs_momentum': [True, False]
    },
    'PolynomialReg': {
        'polynomialfeatures__degree': randint(2, 5),
        'linearregression__fit_intercept': [True, False]
    }
}

# Fine-tune the best model using RandomizedSearchCV
grid_search = RandomizedSearchCV(best_model, param_distributions=param_grids.get(best_model_name, {}),
                                 n_iter=100, cv=kfold, 
                                 scoring='neg_mean_squared_error', n_jobs=-1, 
                                 random_state=42, verbose=1, error_score='raise')

grid_search.fit(X_train, y_train)

# Save the tuned model as 'SOLUTION_model.pkl'
best_model_tuned = grid_search.best_estimator_
store_model_or_pipeline(best_model_tuned, name='SOLUTION_model')

# Save RMSE of the tuned model
best_tuned_rmse = np.sqrt(-grid_search.best_score_)
joblib.dump(best_tuned_rmse, 'saved_objects/SOLUTION_model_rmse.pkl')
print(f"Best RMSE for {best_model_name} after tuning: {best_tuned_rmse:.4f}")



#Cell [7]

# ANALYZE AND TEST THE SOLUTION

# Print the best RMSE achieved after hyperparameter tuning
print(f"\nBest RMSE for {best_model_name} after tuning: {np.sqrt(-grid_search.best_score_):.4f}")

# Make predictions on the test set using the tuned model
y_pred = best_model_tuned.predict(X_test)

# Calculate the RMSE on the test set
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'\nPerformance on test data: RMSE: {test_rmse:.4f}')

# Save the test RMSE for future reference
joblib.dump(test_rmse, 'saved_objects/test_rmse.pkl')



#Cell [8]

# Define a function to find the closest date in the historical data
def find_closest_date(target_date, data):
    try:
        # Try to find the same date in the previous year
        target_date = target_date.replace(year=target_date.year - 1)
    except ValueError:
        # Handle leap year case (February 29)
        target_date = target_date.replace(year=target_date.year - 1, day=28)
    # Find the closest date in the historical data
    closest_date = data['datetime'].iloc[(data['datetime'] - target_date).abs().argsort()[0]]
    return data.loc[data['datetime'] == closest_date].iloc[0]

# Generate future dates for prediction
last_date = raw_data['datetime'].max()
future_dates = pd.date_range(start=last_date + timedelta(days=1), periods=200)
future_data = pd.DataFrame({'datetime': future_dates})

# Fill in other features based on historical data
for col in raw_data.columns:
    if col not in ['datetime', 'tempmax']:
        future_data[col] = future_data['datetime'].apply(lambda x: find_closest_date(x, raw_data)[col])

# Process future data using the same pipeline as training data
future_processed = enhanced_pipeline.transform(future_data)

# Make predictions for future dates
future_pred = best_model_tuned.predict(future_processed)
future_data['predicted_tempmax'] = future_pred

# Save future predictions to a CSV file
future_data[['datetime', 'predicted_tempmax']].to_csv('future_predictions_200days.csv', index=False)

# Visualize future predictions
plt.figure(figsize=(20, 10))
plt.plot(future_data['datetime'], future_data['predicted_tempmax'], label='Predicted Max Temperature', alpha=0.7)
plt.title('Predicted Maximum Temperature for the Next 200 Days')
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()

# Save the plot as an image file
plt.savefig('figures/future_predictions_200days_plot.png', format='png', dpi=300)

# Display the plot
plt.show()

# Print statistics of future predictions
print("\nPredicted Max Temperature:")
print(f"Min: {future_data['predicted_tempmax'].min():.2f}°C")
print(f"Max: {future_data['predicted_tempmax'].max():.2f}°C")
print(f"Avg: {future_data['predicted_tempmax'].mean():.2f}°C")


#Cell [9]

print("\n____________ CONCLUSION ____________")
print("""In this notebook, we have effectively constructed a Machine Learning pipeline that utilizes historical weather data from New York to forecast the maximum temperature (tempmax). 
      We conducted comprehensive coverage of the entire workflow, encompassing data preprocessing, feature engineering, model training, and hyperparameter tuning:""")
print(f"""
   - The best performing model was: {best_model_name}
   - Best model RMSE on test data: {test_rmse:.4f}
   Generated predictions for the next 200 days:
   - The predicted temperatures range from {future_data['predicted_tempmax'].min():.2f}°C to {future_data['predicted_tempmax'].max():.2f}°C.
""")

print("\nPrediction and analysis complete.")